In [1]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers import Dropout
from keras.layers import LSTM

from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback, ModelCheckpoint
import random
import sys
import io

/home/leena/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/leena/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/leena/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/leena/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Taking first 10000 character from data

In [17]:
text = open('/home/leena/Downloads/description_sorted.txt').read()
print('text length in number of characters:', len(text))

text = text[:15000]
print('head of text:')
len(text)

text length in number of characters: 42479302
head of text:


15000

In [18]:
#text = text.lower()

In [19]:
#A set is an unordered collection with no duplicate elements.
#conver back to list, sorts alphanumerically
#list of all unique chars
chars = sorted(list(set(text)))

char_size = len(chars)
print('number of characters:', char_size)
print(chars)


number of characters: 72
['\t', '\n', ' ', '%', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'F', 'G', 'I', 'K', 'L', 'M', 'N', 'O', 'R', 'S', 'T', 'U', 'W', 'Y', '\\', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0']


Now that the book is loaded, we must prepare the data for modeling by the neural network. We cannot model the 
characters directly, instead we must convert the characters to integers.

We can do this easily by first creating a set of all of the distinct characters in the book, then creating a map 
of each character to a unique integer.

In [20]:
#Character to id, and id to character
#dictionary that maps each character to a number and vice versa
char2id = dict((c, i) for i, c in enumerate(chars))
id2char = dict((i, c) for i, c in enumerate(chars))
print(char2id)
print('--------------------------')
print(id2char)

{'\t': 0, '\n': 1, ' ': 2, '%': 3, '&': 4, "'": 5, '(': 6, ')': 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, ';': 23, '=': 24, '?': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'F': 30, 'G': 31, 'I': 32, 'K': 33, 'L': 34, 'M': 35, 'N': 36, 'O': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41, 'W': 42, 'Y': 43, '\\': 44, 'a': 45, 'b': 46, 'c': 47, 'd': 48, 'e': 49, 'f': 50, 'g': 51, 'h': 52, 'i': 53, 'j': 54, 'k': 55, 'l': 56, 'm': 57, 'n': 58, 'o': 59, 'p': 60, 'q': 61, 'r': 62, 's': 63, 't': 64, 'u': 65, 'v': 66, 'w': 67, 'x': 68, 'y': 69, 'z': 70, '\xa0': 71}
--------------------------
{0: '\t', 1: '\n', 2: ' ', 3: '%', 4: '&', 5: "'", 6: '(', 7: ')', 8: ',', 9: '-', 10: '.', 11: '/', 12: '0', 13: '1', 14: '2', 15: '3', 16: '4', 17: '5', 18: '6', 19: '7', 20: '8', 21: '9', 22: ':', 23: ';', 24: '=', 25: '?', 26: 'A', 27: 'B', 28: 'C', 29: 'D', 30: 'F', 31: 'G', 32: 'I', 33: 'K', 34: 'L', 35: 'M', 36: 'N', 37

In [21]:
#Given a probability of each character, return a likely character, one-hot encoded
#our prediction will give us an array of probabilities of each character
#we'll pick the most likely and one-hot encode it
def sample(prediction):
    #Samples are uniformly distributed over the half-open interval 
    r = random.uniform(0,1)
    #store prediction char
    s = 0
    #since length > indices starting at 0
    char_id = len(prediction) - 1
    #for each char prediction probabilty
    for i in range(len(prediction)):
        #assign it to S
        s += prediction[i]
        #check if probability greater than our randomly generated one
        if s >= r:
            #if it is, thats the likely next char
            char_id = i
            break
    #dont try to rank, just differentiate
    #initialize the vector
    char_one_hot = np.zeros(shape=[char_size])
    #that characters ID encoded
    #https://image.slidesharecdn.com/latin-150313140222-conversion-gate01/95/representation-learning-of-vectors-of-words-and-phrases-5-638.jpg?cb=1426255492
    char_one_hot[char_id] = 1.0
    return char_one_hot

In [22]:
#vectorize our data to feed it into model

len_per_section = 100
skip = 1
sections = []
next_chars = []
#fill sections list with chunks of text, every 2 characters create a new 50 
#character long section
#because we are generating it at a character level
for i in range(0, len(text) - len_per_section, skip):
    sections.append(text[i: i + len_per_section])
    next_chars.append(text[i + len_per_section])
#Vectorize input and output
#matrix of section length by num of characters
X = np.zeros((len(sections), len_per_section, char_size))
#label column for all the character id's, still zero
y = np.zeros((len(sections), char_size))
#for each char in each section, convert each char to an ID
#for each section convert the labels to ids 
for i, section in enumerate(sections):
    for j, char in enumerate(section):
        X[i, j, char2id[char]] = 1
    y[i, char2id[next_chars[i]]] = 1
print(y)

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [23]:
X.shape
X[0].shape
y.shape
y[0].shape
X[0].shape

(100, 72)

#The higher the batch size, the more memory space you'll need.
#if you have 1000 training examples, 
#and your batch size is 500, then it will take 2 iterations to complete 1 epoch.
batch_size = 128
#total iterations
max_steps = 10000
#how often to log?
log_every = 1000
#how often to save?
save_every = 1000
#too few and underfitting
#Underfitting occurs when there are too few neurons 
#in the hidden layers to adequately detect the signals in a complicated data set.
#too many and overfitting
hidden_nodes = 100

print('training data size:', len(X))
print('approximate steps per epoch:', int(len(X)/batch_size))

In [57]:
# print(X.shape)
# print(y.shape)

In [58]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [59]:
#optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer="adam")

In [60]:
# checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# cp_callback = tf.keras.callbacks.ModelCheckpoint(
#     checkpoint_path, verbose=1, save_weights_only=True,
#     # Save weights, every 5-epochs.
#     period=5)

In [61]:
#define the checkpoint
filepath="/home/leena/Sohel/training_lstm_wieghts/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min',period=1000)
callbacks_list = [checkpoint]

model.fit(X, y,batch_size=128,epochs=10000,verbose=2,callbacks=callbacks_list)

In [9]:
from keras.models import load_model
model = load_model('/home/leena/LSTM/weights-lstm-34-0.1095.hdf5')

In [34]:
#sentence = 'I plan to make the world a better place '

def random_text_seed_and_generation():


    start_index = random.randint(0, len(text) - 100 - 1)
    test_generated = ''
    sentence = text[start_index: start_index + 100]
    test_generated+= sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(test_generated)
    test_generated = sentence

    for i in range(500):

        x_pred = np.zeros((1,len_per_section,len(chars)))
        for t, char in enumerate(sentence):

            x_pred[0,t,char2id[char]] = 1.


        preds = model.predict(x_pred, verbose=0)[0]
        #one hot encode it
        next_char_one_hot = sample(preds)
        #get the indices of the max values (highest probability)  and convert to char
        next_char = id2char[np.argmax(next_char_one_hot)]
        #add each char to the output text iteratively
        test_generated += next_char
        #update the
        test_X = next_char_one_hot.reshape((1, char_size))
    #a = json.dumps(test_generated)

    return test_generated



In [35]:
nc = random_text_seed_and_generation()

----- Generating with seed: "an\n Blue suitBlue woven blazer in a self-striped pattern, has a notched lapel, single-breasted with"
an\n Blue suitBlue woven blazer in a self-striped pattern, has a notched lapel, single-breasted with

In [38]:
ncoh.shape

(72,)

In [37]:
nc

'Y'

In [34]:
def user_input_text_generation(sentence):
    
    
    test_generated = sentence

    for i in range(500):

        x_pred = np.zeros((1,50,len(chars)))
        for t, char in enumerate(sentence):

            x_pred[0,t,char2id[char]] = 1.


        preds = model.predict(x_pred, verbose=0)[0]
        #one hot encode it
        next_char_one_hot = sample(preds)
        #get the indices of the max values (highest probability)  and convert to char
        next_char = id2char[np.argmax(next_char_one_hot)]
        #add each char to the output text iteratively
        test_generated += next_char
        #update the 
        test_X = next_char_one_hot.reshape((1, char_size))

    return test_generated